In [2]:
# Importing required libraries:
# Requests to make HTTP requests
# BeautifulSoup to parse the HTML content found in the GIC website
# FuzzyWuzzy to match the different names used for banks across different years eg: "State Bank of India" and "SBI"
# Pandas to work with dataframes


!pip install fuzzywuzzy
from fuzzywuzzy import process
import requests
from bs4 import BeautifulSoup
import pandas as pd


# The base URL at which the required files are found to scrape
url = "https://www.gicouncil.in/statistics/industry-statistics/segment-wise-report-on-homepage/"

# Set headers to mimic a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Request the webpage
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extract all text content
valid_months = ['December', 'March', 'June','September']
years = [2020, 2021, 2022, 2023, 2024]
lines = soup.get_text(separator="\n", strip=True).split("\n")
lines2 = [line for line in lines if ("Download Segmentwise" in line) or ('Download Segment wise' in line)]
matched_lines = [line for line in lines2 if any(month in line for month in valid_months)]
files = []

# Print matched lines
if matched_lines:
    for line in matched_lines:
       files.append(line[28:].strip())
files_set = set(files)


report_links = {}
for a in soup.find_all("a", href=True):
    text = a.get_text(strip=True)
    if "Download Segmentwise" in text and any(month in text for month in valid_months):
        report_links[text[27:].strip()] = 'https://www.gicouncil.in'+a["href"]
report_links_set = set(report_links.keys())
missing_months = files_set - report_links_set

print("Missing Months - ")
print(missing_months)

report_links.update({'September 2024': "https://www.gicouncil.in/media/4453/segment_september_2024.xlsx",
'June 2024': "https://www.gicouncil.in/media/4425/segment_june_2024.xlsx",
'December 2024': "https://www.gicouncil.in/media/4480/segment_december_2024-1.xlsx",
'September 2023': "https://www.gicouncil.in/media/4295/segment_september_2023.xlsx",
'September 2018': "https://www.gicouncil.in/media/3143/updated-segment-wise-report-sept-2018.xlsx"})

report_links = dict(sorted(report_links.items()))
report_links_set = set(report_links.keys())
report_links_set



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Missing Months - 
{'June 2024', 'September 2024', 'December 2024', 'September 2023', 'September 2018'}


{'December 2019',
 'December 2020',
 'December 2021',
 'December 2022',
 'December 2023',
 'December 2024',
 'June 2019',
 'June 2020',
 'June 2021',
 'June 2022',
 'June 2023',
 'June 2024',
 'March  2023',
 'March 2019',
 'March 2020',
 'March 2021',
 'March 2022',
 'March 2024',
 'September 2018',
 'September 2020',
 'September 2021',
 'September 2022',
 'September 2023',
 'September 2024'}

In [ ]:
updated_report_links = {}

for key, url in report_links.items():
    parts = key.split()  # Splitting month and year
    if len(parts) == 2:
        month, year = parts
        if month in valid_months and year.isdigit() and int(year) >= 2020:
            new_key = f"{year}_{month}"
            updated_report_links[new_key] = url  # Keep unchanged if not in specified months

# Print transformed dictionary
updated_report_links = dict(sorted(updated_report_links.items()))

updated_report_links

{'2020_December': 'https://www.gicouncil.in/media/3919/new-online-mis_segment-wise-report_december_2020.xlsx',
 '2020_June': 'https://www.gicouncil.in/media/3894/segment-wise-report-for-june-2020.xlsx',
 '2020_March': 'https://www.gicouncil.in/media/3881/segment-wise-report-upto-march-2020_revised.xlsx',
 '2020_September': 'https://www.gicouncil.in/media/3906/segment-wise-report_sept-2020.xlsx',
 '2021_December': 'https://www.gicouncil.in/media/3969/revised-segment-wise-december-2021_xlsx21012022.xlsx',
 '2021_June': 'https://www.gicouncil.in/media/3939/segmentwise-report_june-2021.xlsx',
 '2021_March': 'https://www.gicouncil.in/media/4298/segment-wise-premiums-for-fy-2020-21_updated-2.xlsx',
 '2021_September': 'https://www.gicouncil.in/media/3944/segmentwise-report_september-2021.xlsx',
 '2022_December': 'https://www.gicouncil.in/media/4099/segmentwise-figures-upto-december-2022.xlsx',
 '2022_June': 'https://www.gicouncil.in/media/4069/segment-report-upto-june_2022.xlsx',
 '2022_March

In [ ]:
def data_fetch_and_reframe(url):
  keyword = "Segmentwise"
  xl = pd.ExcelFile(url)
  available_sheets = xl.sheet_names
  sheet_to_use = next((sheet for sheet in available_sheets if keyword.lower() in sheet.lower()), None)
  # print(xl.sheet_names)
  # print(sheet_to_use)

  preview = pd.read_excel(url, sheet_name = sheet_to_use, nrows=10)  # Read first 10 rows to find the correct header row

  for i, row in preview.iterrows():
    if "Fire" in row.values:  # Replace with an actual column name you expect
        header_row = i+1
        break

  dummy = pd.read_excel(url, sheet_name = sheet_to_use, skiprows=header_row)
  dummy.rename(columns={'Unnamed: 0': 'Organization'}, inplace=True)
  dummy.rename(columns={'Insurers': 'Organization'}, inplace=True)

  dummy = dummy[~dummy['Organization'].str.contains('Previous year', case=False, na=False) &
                 ~dummy['Organization'].str.contains("Total", case=False, na=False) &
                 ~dummy['Organization'].str.contains("sub Total", case=False, na=False) &
                 ~dummy['Organization'].str.contains("%", case=False, na=False)]
  dummy = dummy.dropna(subset = list(set(dummy.columns.tolist()) - set(['Organization'])),  how='all')


  reframed_df = dummy.iloc[:,:-4]
  return reframed_df


In [ ]:
df_all = pd.DataFrame(columns=['Organization', 'Fire',
                               'Marine Total', 'Marine  Cargo', 'Marine  Hull', 'Engineering',
                               'Motor Total', 'Motor OD', 'Motor TP',
                               'Health ', 'Aviation ', 'Liability', 'P.A.',
                               'All Other Misc (Crop Insurance + Credit Guarantee+All other misc)', 'Year', 'Quarter'])
print('Generating Dataframw with following reports...')

for key,item in updated_report_links.items():
    print(key)
    try:
        df = data_fetch_and_reframe(item)
        df = df[['Organization', 'Fire', 'Marine Total', 'Marine  Cargo', 'Marine  Hull', 'Engineering',
                'Motor Total', 'Motor OD', 'Motor TP',
                'Health ', 'Aviation ', 'Liability', 'P.A.',
                'All Other Misc (Crop Insurance + Credit Guarantee+All other misc)']]
        df['Year'] = int(key.split('_')[0])
        df['Quarter'] = key.split('_')[1]
        df_all = pd.concat([df_all, df])
    except:
        print(key, 'Fetch error: 400 - Returned data cannot be processed. Check:\n', list(df.columns))


df_all.sort_values('Organization')
df_all


Generating Dataframw with following reports...
2020_December


<ipython-input-4-c9de981dd5b5>:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, df])


2020_June
2020_March
2020_September
2021_December
2021_June
2021_March
2021_September
2022_December


/usr/local/lib/python3.11/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Segmentwise Report'!$A:$R.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
/usr/local/lib/python3.11/dist-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'Segmentwise Report'!$A:$R.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


2022_June
2022_March
2022_September
2023_December
2023_June
2023_March
2023_September
2024_December
2024_June
2024_March
2024_September


,Organization,Fire,Marine Total,Marine Cargo,Marine Hull,Engineering,Motor Total,Motor OD,Motor TP,Health,Aviation,Liability,P.A.,All Other Misc (Crop Insurance + Credit Guarantee+All other misc),Year,Quarter
1,Acko General Insurance Ltd,0.00,0.00,0.00,0.00,0.00,184.43,5.13,28.72,65.14,0.00,23.68,3.66,0.00,2020,December
3,Bajaj Allianz General Insurance Co Ltd,1226.44,123.76,115.53,8.23,161.20,3354.34,197.22,443.63,1617.26,14.24,315.60,148.29,2832.74,2020,December
5,Bharti AXA General Insurance Co Ltd,251.67,56.96,56.96,0.00,29.85,952.91,116.37,155.05,313.09,0.00,35.80,34.31,754.03,2020,December
7,Cholamandalam MS General Insurance Co Ltd,312.76,51.98,51.59,0.39,21.92,2185.85,87.33,74.35,329.72,0.00,13.46,182.40,19.92,2020,December
9,Coco By Navi General Insurance,15.32,0.00,0.00,0.00,0.00,34.92,0.15,5.22,13.99,0.00,0.00,3.33,4.82,2020,December
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,ManipalCigna Health Insurance Co Ltd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,815.91,0.00,0.00,16.09,0.00,2024,September
63,Narayana Health Insurance Ltd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.12,0.00,0.00,0.00,0.00,2024,September
65,Star Health & Allied Insurance Co Ltd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7721.76,0.00,0.00,91.38,0.15,2024,September
71,Agriculture Insurance Co Of India Ltd,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3832.34,2024,September


In [ ]:
# Replace NaN values in the specified column with 'id'
df_all["Organization"] = df_all["Organization"].str.replace(r"\s+", " ", regex=True).str.strip()
def correct_org_name(name):
    match, score = process.extractOne(name, unique_orgs)
    print('Current:', name, '\t', score, '\t New: ',match)
    return match if score > 85 else name  # Adjust threshold (85) if needed

unique_orgs = df["Organization"].unique()
df_all["Organization"] = df_all["Organization"].apply(correct_org_name)
df_all.sort_values(['Organization', 'Year', 'Quarter'], inplace=True, ascending=False)
df_all.reset_index(inplace=True)
# len(df_all)

Current: Acko General Insurance Ltd 	 100 	 New:  Acko General Insurance Ltd
Current: Bajaj Allianz General Insurance Co Ltd 	 100 	 New:  Bajaj Allianz General Insurance Co Ltd
Current: Bharti AXA General Insurance Co Ltd 	 87 	 New:  SBI General Insurance Co Ltd
Current: Cholamandalam MS General Insurance Co Ltd 	 100 	 New:  Cholamandalam MS General Insurance Co Ltd
Current: Coco By Navi General Insurance 	 86 	 New:  Zurich Kotak Mahindra General Insurance Co Ltd
Current: Edelweiss General Insurance Co Ltd 	 87 	 New:  SBI General Insurance Co Ltd
Current: Future Generali India Insurance Co Ltd 	 100 	 New:  Future Generali India Insurance Co Ltd
Current: Go Digit General Insurance Ltd 	 100 	 New:  Go Digit General Insurance Ltd
Current: HDFC Ergo General Insurance Co Ltd 	 100 	 New:  HDFC Ergo General Insurance Co Ltd
Current: ICICI Lombard General Insurance Co Ltd 	 100 	 New:  ICICI Lombard General Insurance Co Ltd
Current: IFFCO-Tokio General Insurance Co Ltd 	 100 	 New:  IF

In [ ]:
df_all["Term"] = pd.to_datetime("30" +" "+ df_all["Quarter"] +" "+ df_all["Year"].astype(str), format="mixed")

df_all.columns

Index(['index', 'Organization', 'Fire', 'Marine Total', 'Marine  Cargo',
       'Marine  Hull', 'Engineering', 'Motor Total', 'Motor OD', 'Motor TP',
       'Health ', 'Aviation ', 'Liability', 'P.A.',
       'All Other Misc (Crop Insurance + Credit Guarantee+All other misc)',
       'Year', 'Quarter', 'Term'],
      dtype='object')

In [ ]:
df_all['Grand_Total_By_Org_By_Term'] = df_all['Fire'] + df_all['Marine  Cargo'] + df_all['Marine  Hull'] +  df_all['Engineering'] + df_all['Motor OD'] + df_all['Motor TP'] + df_all['Health '] +df_all['Aviation '] + df_all['Liability'] + df_all['P.A.'] + df_all['All Other Misc (Crop Insurance + Credit Guarantee+All other misc)']

In [ ]:
df_all.sort_values(['Organization', 'Term'], inplace=True, ascending=True)
df_all.tail(20)

,index,Organization,Fire,Marine Total,Marine Cargo,Marine Hull,Engineering,Motor Total,Motor OD,Motor TP,Health,Aviation,Liability,P.A.,All Other Misc (Crop Insurance + Credit Guarantee+All other misc),Year,Quarter,Term,Grand_Total_By_Org_By_Term
14,9,Zurich Kotak Mahindra General Insurance Co Ltd,-0.48,0.00,0.00,0.0,0.00,6.22,2.07,4.15,1.15,0.0,0.00,0.17,0.65,2021,June,2021-06-30,7.71
15,23,Zurich Kotak Mahindra General Insurance Co Ltd,9.75,0.10,0.10,0.0,0.50,52.98,28.98,24.00,48.01,0.0,0.14,6.12,1.40,2021,June,2021-06-30,119.00
16,63,Zurich Kotak Mahindra General Insurance Co Ltd,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2021,June,2021-06-30,0.00
12,21,Zurich Kotak Mahindra General Insurance Co Ltd,22.34,0.72,0.72,0.0,1.30,129.72,73.56,56.16,103.58,0.0,0.28,16.13,3.60,2021,September,2021-09-30,277.67
17,21,Zurich Kotak Mahindra General Insurance Co Ltd,36.97,2.72,2.72,0.0,2.44,259.34,155.62,103.72,162.41,0.0,0.41,27.40,8.88,2021,December,2021-12-30,500.57
18,59,Zurich Kotak Mahindra General Insurance Co Ltd,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2021,December,2021-12-30,0.00
19,63,Zurich Kotak Mahindra General Insurance Co Ltd,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2021,December,2021-12-30,0.00
20,67,Zurich Kotak Mahindra General Insurance Co Ltd,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,2021,December,2021-12-30,0.00
9,21,Zurich Kotak Mahindra General Insurance Co Ltd,50.95,4.90,4.90,0.0,3.52,388.03,235.21,152.81,236.27,0.0,0.62,39.26,18.92,2022,March,2022-03-30,742.46
10,21,Zurich Kotak Mahindra General Insurance Co Ltd,19.20,4.06,4.06,0.0,2.04,100.71,54.24,46.46,72.48,0.0,0.17,11.29,7.57,2022,June,2022-06-30,217.51


In [ ]:
df_all.to_csv("data.csv", index=False)